# 12. Improving Simple CNN

In [1]:
import torch
import torch.nn as nn

import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.utils.data as Data
import torch.nn.init as init

from torch.autograd import Variable
import numpy as np
import os

## 12.1 Preparing Custom Data

In [2]:
img_dir = "./image"
img_data = dsets.ImageFolder(img_dir, transforms.Compose([
            transforms.Grayscale(),
#           Data Augmentation
#           transforms.RandomRotation(15)
#           transforms.CenterCrop(28),
#           transforms.Lambda(lambda x: x.rotate(15)),
    
#           Data Nomalization
#           transforms.Normalize(mean=(0.5,), std=(0.5,))
            transforms.ToTensor()
            ]))

#https://pytorch.org/docs/stable/torchvision/transforms.html

print(img_data.classes)
print(img_data.class_to_idx)

['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
{'ㄱ': 0, 'ㄲ': 1, 'ㄴ': 2, 'ㄷ': 3, 'ㄸ': 4, 'ㄹ': 5, 'ㅁ': 6, 'ㅂ': 7, 'ㅃ': 8, 'ㅅ': 9, 'ㅆ': 10, 'ㅇ': 11, 'ㅈ': 12, 'ㅉ': 13, 'ㅊ': 14, 'ㅋ': 15, 'ㅌ': 16, 'ㅍ': 17, 'ㅎ': 18, 'ㅏ': 19, 'ㅐ': 20, 'ㅑ': 21, 'ㅒ': 22, 'ㅓ': 23, 'ㅔ': 24, 'ㅕ': 25, 'ㅖ': 26, 'ㅗ': 27, 'ㅘ': 28, 'ㅙ': 29, 'ㅛ': 30, 'ㅜ': 31, 'ㅝ': 32, 'ㅞ': 33, 'ㅟ': 34, 'ㅠ': 35, 'ㅡ': 36, 'ㅢ': 37, 'ㅣ': 38}


In [3]:
batch_size = 100
font_num = 720

In [4]:
def train_test_split(data, train_ratio, stratify, stratify_num, batch_size) :
    
    length = len(data)
    
    if stratify :
        label_num = int(len(data)/stratify_num)
        cut = int(stratify_num*train_ratio)
        train_indices = np.random.permutation(np.arange(stratify_num))[:cut]
        test_indices = np.random.permutation(np.arange(stratify_num))[cut:]
        
        for i in range(1, label_num) :
            train_indices = np.concatenate((train_indices, np.random.permutation(np.arange(stratify_num))[:cut] + stratify_num*i))
            test_indices = np.concatenate((test_indices, np.random.permutation(np.arange(stratify_num))[cut:] + stratify_num*i))
        
    else :
        cut = int(len(data)*train_ratio)
        train_indices = np.random.permutation(np.arange(length))[:cut]
        test_indices = np.random.permutation(np.arange(length))[cut:]
        
    train_loader = Data.DataLoader(data, batch_size=batch_size, shuffle=False, sampler = train_indices, num_workers=1, drop_last = True)
    test_loader = Data.DataLoader(data, batch_size=batch_size, shuffle=False, sampler = test_indices, num_workers=1, drop_last = True)

    return train_loader, test_loader, len(train_indices), len(test_indices)

In [5]:
train_loader, test_loader, train_num, test_num = train_test_split(img_data, 0.8, True, font_num, batch_size)

## 12.2 Define Model

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            #Dropout
            #nn.Dropout2d(0.2),
            nn.Conv2d(16,32,5),
            #Batch Nomalization
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*5*5,100),
            nn.ReLU(),
            nn.Linear(100,39)
        )
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
#                 Weight Initialization
#                 init.xavier_normal(m.weight.data)
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)
        
            elif isinstance(m, nn.Linear):
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)                
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)

        return out

model = CNN().cuda()

In [7]:
loss = nn.CrossEntropyLoss() # Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Momentum & Weight Regularization(L2)
# optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-5)

In [8]:
num_epochs = 200

In [9]:
# Learning Rate Scheduler
#scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma= 0.99)
#scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[10,30,80], gamma= 0.1)
#scheduler = lr_scheduler.ExponentialLR(optimizer, gamma= 0.99)
#scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')

for epoch in range(num_epochs):
#     scheduler.step()
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        
        X = Variable(batch_images).cuda()
        Y = Variable(batch_labels).cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, train_num//batch_size, cost.data[0]))

Epoch [1/200], lter [100/224] Loss: 4.4944
Epoch [1/200], lter [200/224] Loss: 3.8000
Epoch [2/200], lter [100/224] Loss: 3.6617
Epoch [2/200], lter [200/224] Loss: 3.6310
Epoch [3/200], lter [100/224] Loss: 3.6120
Epoch [3/200], lter [200/224] Loss: 2.9988
Epoch [4/200], lter [100/224] Loss: 3.9975
Epoch [4/200], lter [200/224] Loss: 2.1550
Epoch [5/200], lter [100/224] Loss: 3.9255
Epoch [5/200], lter [200/224] Loss: 1.4258
Epoch [6/200], lter [100/224] Loss: 2.9334
Epoch [6/200], lter [200/224] Loss: 3.6822
Epoch [7/200], lter [100/224] Loss: 3.6790
Epoch [7/200], lter [200/224] Loss: 2.4829
Epoch [8/200], lter [100/224] Loss: 2.0596
Epoch [8/200], lter [200/224] Loss: 1.7899
Epoch [9/200], lter [100/224] Loss: 1.8682
Epoch [9/200], lter [200/224] Loss: 1.2327
Epoch [10/200], lter [100/224] Loss: 2.0146
Epoch [10/200], lter [200/224] Loss: 1.8768
Epoch [11/200], lter [100/224] Loss: 2.7540
Epoch [11/200], lter [200/224] Loss: 1.5010
Epoch [12/200], lter [100/224] Loss: 3.4090
Epoch 

Epoch [94/200], lter [200/224] Loss: 0.0000
Epoch [95/200], lter [100/224] Loss: 0.0000
Epoch [95/200], lter [200/224] Loss: 0.0000
Epoch [96/200], lter [100/224] Loss: 0.0000
Epoch [96/200], lter [200/224] Loss: 0.0000
Epoch [97/200], lter [100/224] Loss: 0.0000
Epoch [97/200], lter [200/224] Loss: 0.0000
Epoch [98/200], lter [100/224] Loss: 0.0000
Epoch [98/200], lter [200/224] Loss: 0.0000
Epoch [99/200], lter [100/224] Loss: 0.0000
Epoch [99/200], lter [200/224] Loss: 0.0000
Epoch [100/200], lter [100/224] Loss: 0.0000
Epoch [100/200], lter [200/224] Loss: 0.0000
Epoch [101/200], lter [100/224] Loss: 0.0000
Epoch [101/200], lter [200/224] Loss: 0.0000
Epoch [102/200], lter [100/224] Loss: 0.0000
Epoch [102/200], lter [200/224] Loss: 0.0000
Epoch [103/200], lter [100/224] Loss: 0.0000
Epoch [103/200], lter [200/224] Loss: 0.0000
Epoch [104/200], lter [100/224] Loss: 0.0000
Epoch [104/200], lter [200/224] Loss: 0.0000
Epoch [105/200], lter [100/224] Loss: 0.0000
Epoch [105/200], lter

Epoch [186/200], lter [100/224] Loss: 0.0000
Epoch [186/200], lter [200/224] Loss: 0.0000
Epoch [187/200], lter [100/224] Loss: 0.0000
Epoch [187/200], lter [200/224] Loss: 0.0000
Epoch [188/200], lter [100/224] Loss: 0.0000
Epoch [188/200], lter [200/224] Loss: 0.0000
Epoch [189/200], lter [100/224] Loss: 0.2226
Epoch [189/200], lter [200/224] Loss: 0.0112
Epoch [190/200], lter [100/224] Loss: 0.1392
Epoch [190/200], lter [200/224] Loss: 0.0530
Epoch [191/200], lter [100/224] Loss: 0.1792
Epoch [191/200], lter [200/224] Loss: 0.0014
Epoch [192/200], lter [100/224] Loss: 0.0715
Epoch [192/200], lter [200/224] Loss: 0.0031
Epoch [193/200], lter [100/224] Loss: 0.3028
Epoch [193/200], lter [200/224] Loss: 0.0027
Epoch [194/200], lter [100/224] Loss: 0.0382
Epoch [194/200], lter [200/224] Loss: 0.0005
Epoch [195/200], lter [100/224] Loss: 0.0019
Epoch [195/200], lter [200/224] Loss: 0.0003
Epoch [196/200], lter [100/224] Loss: 0.0011
Epoch [196/200], lter [200/224] Loss: 0.0003
Epoch [197

## 12.3 Test Model

In [10]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = Variable(images).cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of test images: %f %%' % (100 * correct / total))

Accuracy of test images: 96.517857 %
